**Capítulo 6: Arvores de Decisão**

### **Resumo**
O Capítulo 6 do livro "Hands-On Machine Learning with Scikit-Learn & TensorFlow" foca nas **Árvores de Decisão (Decision Trees)**, que são algoritmos de Machine Learning versáteis capazes de realizar tarefas de classificação e regressão, incluindo tarefas de saída múltipla. São componentes fundamentais dos Random Forests.

Os conceitos mais importantes detalhados no capítulo são:

*   **Treinamento e Visualização de uma Árvore de Decisão**:
    *   As árvores de decisão são treinadas para fazer previsões.
    *   Para classificar uma flor Iris, por exemplo, a árvore começa no nó raiz e faz perguntas sequenciais (e.g., "comprimento da pétala é menor que 2.45 cm?") até chegar a um nó folha que indica a classe prevista.
    *   Uma grande vantagem é que **requerem muito pouca preparação de dados**, não sendo necessário o *feature scaling* ou o *centering*.

*   **Fazendo Previsões**:
    *   Os atributos de um nó incluem:
        *   `samples`: Conta quantas instâncias de treinamento se aplicam ao nó.
        *   `value`: Indica quantas instâncias de treinamento de cada classe se aplicam a esse nó.
        *   `gini`: Mede a impureza do nó. Um nó é "puro" se todas as instâncias pertencerem à mesma classe (gini=0). A fórmula da impureza de Gini para o i-ésimo nó é `Gi = 1 - Σk=1^n (pi,k)^2`, onde `pi,k` é a proporção de instâncias da classe k no i-ésimo nó.
    *   O Scikit-Learn utiliza o algoritmo **CART (Classification And Regression Tree)**, que produz apenas árvores binárias, onde os nós não-folha sempre têm dois filhos (perguntas com respostas sim/não).
    *   As **árvores de decisão são modelos "caixa branca" (white box)**, o que significa que suas decisões são intuitivas e fáceis de interpretar, fornecendo regras de classificação simples.

*   **Estimando Probabilidades de Classe**:
    *   Uma árvore de decisão pode estimar a probabilidade de uma instância pertencer a uma classe específica. Ela encontra o nó folha correspondente à instância e retorna a proporção de instâncias de treinamento daquela classe no nó.

*   **O Algoritmo de Treinamento CART**:
    *   O algoritmo **CART** (Classification And Regression Tree) divide o conjunto de treinamento em dois subconjuntos usando uma única *feature* `k` e um *threshold* `tk` (e.g., "comprimento da pétala ≤ 2.45 cm").
    *   Ele busca o par `(k, tk)` que produz os subconjuntos mais puros (ponderados pelo seu tamanho). A função de custo que o algoritmo minimiza para classificação é `J(k, tk) = (m_left/m) * G_left + (m_right/m) * G_right`, onde `G_left/right` mede a impureza do subconjunto esquerdo/direito e `m_left/right` é o número de instâncias nesse subconjunto.
    *   É um **algoritmo guloso (greedy algorithm)**, pois busca uma divisão ótima no nível atual, mas não garante a solução ótima global em vários níveis abaixo.
    *   A **complexidade computacional** do treinamento de uma Árvore de Decisão é `O(n × m log(m))`, onde `n` é o número de *features* e `m` é o número de instâncias de treinamento.

*   **Impureza de Gini ou Entropia?**:
    *   Por padrão, o Scikit-Learn usa a **medida de impureza de Gini**.
    *   Alternativamente, pode-se usar a **entropia**, definida como `Hi = - Σk=1^n (pi,k * log(pi,k))`. A entropia mede o desordem molecular ou o conteúdo médio de informação de uma mensagem.
    *   Na maioria das vezes, **não há uma grande diferença** entre usar Gini ou entropia, pois levam a árvores semelhantes. Gini é ligeiramente mais rápida de calcular. No entanto, Gini tende a isolar a classe mais frequente em seu próprio ramo, enquanto a entropia tende a produzir árvores mais equilibradas.

*   **Hiperparâmetros de Regularização**:
    *   Para evitar o sobreajuste (*overfitting*), pode-se restringir a forma da Árvore de Decisão usando hiperparâmetros como:
        *   `max_depth`: A profundidade máxima da árvore.
        *   `min_samples_split`: O número mínimo de amostras que um nó deve ter antes de poder ser dividido.
        *   `min_samples_leaf`: O número mínimo de amostras que um nó folha deve ter.
        *   `min_weight_fraction_leaf`: Similar a `min_samples_leaf`, mas expresso como uma fração do total de instâncias ponderadas.
        *   `max_leaf_nodes`: O número máximo de nós folha.
        *   `max_features`: O número máximo de *features* avaliadas para divisão em cada nó.
    *   Aumentar os hiperparâmetros `min_*` ou reduzir os `max_*` **regularizará o modelo**.

*   **Regressão com Árvores de Decisão**:
    *   Para tarefas de regressão, as Árvores de Decisão dividem o conjunto de treinamento para minimizar o **Erro Quadrático Médio (MSE)**, em vez da impureza.
    *   O valor previsto para cada região é a média dos valores alvo das instâncias naquela região.
    *   Assim como na classificação, as Árvores de Decisão são **propensas ao sobreajuste** em tarefas de regressão e precisam ser regularizadas.

*   **Instabilidade**:
    *   Árvores de Decisão são **sensíveis à rotação do conjunto de treinamento**, pois preferem limites de decisão ortogonais. A rotação dos dados pode levar a limites de decisão desnecessariamente complexos, o que prejudica a generalização. Uma solução é usar PCA para melhor orientação dos dados.
    *   Também são **muito sensíveis a pequenas variações nos dados de treinamento**, o que pode levar a árvores muito diferentes. Random Forests podem mitigar essa instabilidade.

### **Implementação**

In [15]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeRegressor
import math
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import ShuffleSplit
from scipy.stats import mode

Árvores de decisão

In [2]:
iris = load_iris()
X = iris.data[:, 2:]
y = iris.target

In [3]:
tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2, random_state=42)

Fazendo previsões

In [4]:
tree_clf.predict_proba([[5,1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [5]:
tree_clf.predict([[5,1.5]])

array([1])

Regressão

In [8]:
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

DecisionTreeRegressor(max_depth=2)

### **Exercícios**

1. Qual é a profundidade aproximada de uma árvore de Decisão treinada (sem restrições) em um conjunto com 1 milhão de instâncias?

In [10]:
n = 1_000_000
profundidade_aproximada = math.ceil(math.log2(n))
print(f"Profundidade aproximada: {profundidade_aproximada}")

Profundidade aproximada: 20


2. O coeficiente de Gini de um nó geralmente é menor ou maior do que os seus pais? Ele é geralmente menor/menor, ou sempre menor/maior?

O coeficiente de Gini de um nó filho geralmente é menor do que o de seu nó pai. Isso ocorre porque, ao dividir um nó, o algoritmo de árvore de decisão busca criar partições mais "puras", ou seja, com menor impureza (menor Gini).
No entanto, o coeficiente de Gini dos filhos pode ser igual ao do pai se a divisão não aumentar a pureza, mas nunca será maior. Portanto, o coeficiente de Gini dos filhos é geralmente menor ou igual ao dos pais, mas nunca maior.

3. É uma boa ideia tentar diminuir seu max_depth se uma árvore de decisão estiver sobreajustando ao conjunto de treinamento?

Sim, é uma boa ideia diminuir o parâmetro max_depth se uma árvore de decisão estiver sobreajustando (overfitting) ao conjunto de treinamento.
Ao limitar a profundidade máxima da árvore, você reduz sua complexidade e, consequentemente, sua capacidade de memorizar detalhes e ruídos do conjunto de treinamento. Isso ajuda a melhorar a generalização do modelo para novos dados.

4. É uma boa ideia tentar dimensionar as características de entrada se uma árvore de decisão estiver se subajustando ao conjunto de treinamento?

Não, não é necessário dimensionar (escalar) as características de entrada quando se utiliza árvores de decisão, mesmo que o modelo esteja se subajustando (underfitting) ao conjunto de treinamento.
Árvores de decisão são invariantes a escala das variáveis, pois as divisões são feitas com base em limiares dos próprios valores das características. Para combater o subajuste, é melhor aumentar a complexidade do modelo (por exemplo, aumentando max_depth, reduzindo min_samples_split, etc.).

5. Se treinar uma árvore de decisão em um conjunto de treinamento contendo 1 milhão de instâncias demora 1 hora, aproximadamente quando tempo demorará para treinas outra árvore de decisão em um conjunto de treinamento contendo 10 milhoes de instâncias?

O tempo de treinamento de uma árvore de decisão cresce mais do que linearmente com o número de instâncias, pois a complexidade é aproximadamente O(n × log n), onde n é o número de instâncias.
Se para 1 milhão de instâncias leva 1 hora, para 10 milhões de instâncias o tempo será aproximadamente:
Tempo ≈ 1 hora × (10.000.000 × log₂10.000.000) / (1.000.000 × log₂1.000.000) = (10.000.000 × 24) / (1.000.000 × 20) = 12
Portanto, treinar a árvore com 10 milhões de instâncias levará cerca de 12 horas.

6. Se o seu conjunto de treinamento contém 100 mil instâncias, a configuração presort=True acelerará o treinamento?

Não, a configuração presort=True não acelerará o treinamento em conjuntos de dados grandes, como 100 mil instâncias. Na verdade, pode até deixar o treinamento mais lento.
O parâmetro presort=True faz com que as características sejam pré-ordenadas para cada divisão, o que só é vantajoso para conjuntos de dados pequenos. Para conjuntos grandes, o custo de pré-ordenar os dados supera os benefícios, tornando o treinamento mais demorado.

7. Treine e ajuste uma árvore de decisão para o conjunto de dados de luas.

        A. Gere um conjunto de dados de luas utilizando make_moons(n_samples=10000, noise=0.4)

        B. Com a utilização do train_test_split(), divida em um conjunto de treinamento e um conjunto de testes

        C. Utilize a pesquisa de grade com validação cruzada (com a ajuda da classe GridSearchCV) para encontrar bons valores de hiperparâmetros para um DecisionTreeClassifier.

Dica: Tente vários valores para max_leaf_nodes

In [14]:
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'max_leaf_nodes': list(range(2, 51)),
    'min_samples_split': [2, 3, 4]
}
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

print('Melhores hiperparâmetros:', grid_search.best_params_)
print('Acurácia no conjunto de teste:', grid_search.score(X_test, y_test))

Melhores hiperparâmetros: {'max_leaf_nodes': 17, 'min_samples_split': 2}
Acurácia no conjunto de teste: 0.8695


8. Cultive uma floresta.
   
        a. Continuando com o exercício anterior, gere mil subconjuntos do conjunto de treinamento, cada um contendo 100 instâncias selecionadas aleatóriamente. Dica: Você pode utilizar ShuffleSplit do sklearn para isso.
        b. Treine uma árvore de decisão em cada subconjunto utilizando os melhores valores dos hiperparâmetros encontrados anteriormente. Avalie essas mil árvores de decisão no conjunto de testes. Uma vez treinadas em conjuntos menores, essas árvores de decisão provavelmente terão um desempenho pior do que a primeira, alcançando apenas 80% de acurácia.
        c. Agora vem a mágica. Gere as previsões das mil árvores de decisão e mantenha apenas a previsão mais frequente para cada instância do conjunto de testes (você pode utilizar a função mode() do SciPy para isso). Isso lhe dá previsões dos votos majoritários sobre o conjunto de testes.
        d. Avalie estas previsões no conjunto de teste: você deve obter uma acurácia ligeiramente maior que o seu primeiro modelo (cerca de 0,5 a 1,5% a mais).

In [18]:
n_subsets = 1000
subset_size = 100
shuffle_split = ShuffleSplit(n_splits=n_subsets, test_size=len(X_train) - subset_size, random_state=42)

subsets = []
for train_idx, _ in shuffle_split.split(X_train):
    subsets.append((X_train[train_idx], y_train[train_idx]))

trees = []
for X_subset, y_subset in subsets:
    tree = DecisionTreeClassifier(max_leaf_nodes=grid_search.best_params_['max_leaf_nodes'],
                                  min_samples_split=grid_search.best_params_['min_samples_split'],
                                  random_state=42)
    tree.fit(X_subset, y_subset)
    trees.append(tree)

predictions = np.array([tree.predict(X_test) for tree in trees])
majority_vote = mode(predictions, axis=0).mode[0]

accuracy = np.mean(majority_vote == y_test)
print(f"Acurácia das previsões por votação majoritária: {accuracy:.2f}")

Acurácia das previsões por votação majoritária: 0.49
